#Лабораторная работа №3. Применение сети Хопфилда в  машинном обучении

## Цель работы

Изучить принципы работы сети Хопфилда, реализовать модель
и провести эксперименты, демонстрирующие её применение в задачах машинного
обучения.


##Вариант 2

Вариант 2:
Улучшение представления данных в задачах классификации

Датасет: UCI Machine Learning Repository – "Mushroom" (данные о
съедобности грибов, содержащие 8124 образца с 22 категориальными признаками).

Задание:
1. Загрузить датасет "Mushroom" и закодировать бинарные признаки
(например, используя one-hot encoding).
2. Обучить сеть Хопфилда на части данных и использовать её выходное
представление как новую форму входных данных.
3. Провести сравнение классификации (используя логистическую регрессию
или SVM) на исходных данных и после обработки сетью Хопфилда.
4. Оценить, влияет ли использование сети Хопфилда на точность
классификации.


##Ход выполнения работы

Для начала импортируется библиотека pandas, к который мы будем обращаться через alias pd. Считывается датасет.

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/mushrooms.csv')
df

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,e,k,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,b,c,l
8120,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,n,o,p,b,v,l
8121,e,f,s,n,f,n,a,c,b,n,...,s,o,o,p,o,o,p,b,c,l
8122,p,k,y,n,f,y,f,c,n,b,...,k,w,w,p,w,o,e,w,v,l


Этот набор данных содержит описания гипотетических образцов, соответствующих 23 видам жаброногих грибов из родов Agaricus и Lepiota, взятых из «Полевого справочника по грибам Северной Америки» (1981) Общества Одюбона. Каждый вид идентифицирован как определенно съедобный, определенно ядовитый, или как неизвестный съедобный и не рекомендуемый. Последний класс был объединен с ядовитым. В путеводителе четко сказано, что не существует простого правила для определения съедобности гриба; нет такого правила, как «три листочка, пусть будет» для ядовитого дуба и плюща.
Описание взято с сайта Kaggle.

Данные проверяются на пропуски.

In [ ]:
print(df.nunique())


class                        2
cap-shape                    6
cap-surface                  4
cap-color                   10
bruises                      2
odor                         9
gill-attachment              2
gill-spacing                 2
gill-size                    2
gill-color                  12
stalk-shape                  2
stalk-root                   5
stalk-surface-above-ring     4
stalk-surface-below-ring     4
stalk-color-above-ring       9
stalk-color-below-ring       9
veil-type                    1
veil-color                   4
ring-number                  3
ring-type                    5
spore-print-color            9
population                   6
habitat                      7
dtype: int64


In [ ]:
print(df.isnull().sum())

class                       0
cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64


Все признаки в датасете являются категориальными.
Целевая переменная - class (съедобный e или ядовитый p).
Для преобразования категориальных признаков в числовой формат используется метод pd.get_dummies. Этот метод создаёт бинарные признаки для каждой категории.

In [ ]:
df_encoded = pd.get_dummies(df, drop_first=True)
df_encoded.head()

,class_p,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_g,cap-surface_s,cap-surface_y,cap-color_c,...,population_n,population_s,population_v,population_y,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
0,True,False,False,False,False,True,False,True,False,False,...,False,True,False,False,False,False,False,False,True,False
1,False,False,False,False,False,True,False,True,False,False,...,True,False,False,False,True,False,False,False,False,False
2,False,False,False,False,False,False,False,True,False,False,...,True,False,False,False,False,False,True,False,False,False
3,True,False,False,False,False,True,False,False,True,False,...,False,True,False,False,False,False,False,False,True,False
4,False,False,False,False,False,True,False,True,False,False,...,False,False,False,False,True,False,False,False,False,False


Данные разделяются на признаки и целевую переменную

In [ ]:
X = df_encoded.drop('class_p', axis=1)
y = df_encoded['class_p']

print("Признаки (X):", X.shape)
print("Целевая переменная (y):", y.shape)

Признаки (X): (8124, 95)
Целевая переменная (y): (8124,)


Данные стандартизируются для того чтобы привести все признаки к единому масштабу, что улучшит работу алгоритма

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

После данные разделяются на две выборки, обучающую и тестовую

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Обучающая выборка (X_train):", X_train.shape)
print("Тестовая выборка (X_test):", X_test.shape)

Обучающая выборка (X_train): (6499, 95)
Тестовая выборка (X_test): (1625, 95)


Далее реализуется сама модель

- train(patterns) - для каждого входного паттерна обновляются веса по формуле W = X X.T, а затем нормируются.
- recall(input_pattern) -  восстановление (ассоциативный поиск). На каждом шаге обновляется входной вектор, умноженный на матрицу весов, с применением функции torch.sign(), которая превращает элементы в -1 или 1.

Обучение модели
- X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)`: преобразует данные в тензор.
- hopfield_net = HopfieldNetwork(size=X_train.shape[1]) - создаёт модель с числом входов, равным количеству признаков в X_train.
- hopfield_net.train(X_train_tensor): обучает сеть на множестве паттернов.

Далее патерн восстанавливается

In [ ]:
import torch
import torch.nn as nn

class HopfieldNetwork(nn.Module):
    def __init__(self, size):
        super(HopfieldNetwork, self).__init__()
        self.size = size #Строка из призанков n
        self.weights = torch.zeros(size, size) #Матрица M * n

    def train(self, patterns):
        for pattern in patterns:
            pattern = pattern.view(-1, 1)
            self.weights += torch.mm(pattern, pattern.t()) #Весса с транспонированной матрицей
        self.weights /= len(patterns) # 1/n
        self.weights.fill_diagonal_(0)  # Нейроны не связаны сами с собой, диагональ зануляем

    def recall(self, input_pattern, max_iter=100):
        pattern = input_pattern.clone()
        for _ in range(max_iter):
            pattern = torch.sign(torch.mv(self.weights, pattern))
        return pattern

# Преобразуем данные в тензоры
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)

hopfield_net = HopfieldNetwork(size=X_train.shape[1])
hopfield_net.train(X_train_tensor)

# Пример восстановления паттерна
input_pattern = X_train_tensor[0]
restored_pattern = hopfield_net.recall(input_pattern)
print("Паттерн до восстановления:", input_pattern)
print("Восстановленный паттерн:", restored_pattern)

Паттерн до восстановления: tensor([0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0.])
Восстановленный паттерн: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1.])


Оценивается точность логистической регрессии на исходных данных и на данных, прошедших через сеть Хопфилда

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)

y_pred = log_reg.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Точность на исходных данных:", accuracy)

import numpy as np
import torch

def transform_with_hopfield(net, data):
    transformed_data = []
    for sample in data:
        sample_tensor = torch.tensor(sample, dtype=torch.float32)
        transformed_sample = net.recall(sample_tensor).numpy()
        transformed_data.append(transformed_sample)
    return np.array(transformed_data)

X_train_hopfield = transform_with_hopfield(hopfield_net, X_train.values)
X_test_hopfield = transform_with_hopfield(hopfield_net, X_test.values)

log_reg_hopfield = LogisticRegression(max_iter=1000)
log_reg_hopfield.fit(X_train_hopfield, y_train)

y_pred_hopfield = log_reg_hopfield.predict(X_test_hopfield)
accuracy_hopfield = accuracy_score(y_test, y_pred_hopfield)
print("Точность на данных после обработки сетью Хопфилда:", accuracy_hopfield)


Точность на исходных данных: 1.0
Точность на данных после обработки сетью Хопфилда: 0.5187692307692308


Хопфилд изменяет категориальные признаки, а он не предназначен для работы с такими данными. Вот с картинками было бы другое дело.

# Вывод



В ходе выполнения лабораторной работы были проведены исследования влияния сети Хопфилда на представление данных и точность классификации. Логистическая регрессия, обученная на исходных данных, достигла точности 1.0, что свидетельствует о хорошо разделимых классах и наличии явных закономерностей в признаках.  

Сеть Хопфилда использовалась для предварительной обработки данных. Она была обучена на бинаризованных признаках, преобразованных в -1 и 1, и пыталась восстановить паттерны на основе обученных весов. Однако это привело к изменению структуры данных, что могло внести искажения в их исходное представление.  

После обработки сетью Хопфилда точность логистической регрессии снизилась до 0.5188. Это может быть связано с тем, что преобразование изменило исходные закономерности, внесло шум и ухудшило разделимость классов. Так как сеть Хопфилда предназначена для хранения и восстановления паттернов, а не для улучшения классификации, её применение в данной задаче оказалось неэффективным.  

Таким образом, использование сети Хопфилда в данном эксперименте не привело к улучшению классификации, а напротив, снизило точность модели. Это подчеркивает важность выбора подходящих методов предобработки данных с учётом специфики задачи.